In [2]:
# Mount to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import Libraries
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

In [4]:
# Reading Data - 'Music Info.csv'
music_info = pd.read_csv("/content/drive/MyDrive/Music Info.csv")
music_info.head(5)

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


In [5]:
# Reading Data - 'User Listening History.csv'
listening_info = pd.read_csv("/content/drive/MyDrive/User Listening History.csv")
listening_info.head(5)

,track_id,user_id,playcount
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1


In [6]:
# merging two dataframes - 'Music Info.csv', 'User Listening History.csv'
left = music_info.set_index('track_id')
right = listening_info.set_index('track_id')
music_listening_info = left.join(right)
music_listening_info.head(5)

,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,user_id,playcount
track_id,,,,,,,,,,,,,,,,,,,,,
TRAAADT12903CCC339,Andalucia,Pink Martini,https://p.scdn.co/mp3-preview/37d9c700532305fc...,16uYYyMYpiJsUC9JzZX4Zk,"jazz, french, lounge",NaN,1997,219360,0.714,0.521,...,1,0.0363,0.727,0.825000,0.074,0.683,92.530,3,NaN,NaN
TRAAAED128E0783FAB,It's About Time,Jamie Cullum,https://p.scdn.co/mp3-preview/9bb7b58f848a742b...,00ZCW9lUa2bqwpF6rhtaMI,"jazz, piano, chill",Jazz,2004,247000,0.565,0.507,...,1,0.0541,0.702,0.000066,0.129,0.454,152.462,4,cf7bd4b5b398b3e150cf262d79147312a69b96ac,9.0
TRAAAED128E0783FAB,It's About Time,Jamie Cullum,https://p.scdn.co/mp3-preview/9bb7b58f848a742b...,00ZCW9lUa2bqwpF6rhtaMI,"jazz, piano, chill",Jazz,2004,247000,0.565,0.507,...,1,0.0541,0.702,0.000066,0.129,0.454,152.462,4,043d81932e75d5749ed5758d6420506e7bc457a5,3.0
TRAAAED128E0783FAB,It's About Time,Jamie Cullum,https://p.scdn.co/mp3-preview/9bb7b58f848a742b...,00ZCW9lUa2bqwpF6rhtaMI,"jazz, piano, chill",Jazz,2004,247000,0.565,0.507,...,1,0.0541,0.702,0.000066,0.129,0.454,152.462,4,515e1ab04c00859de983cacf35f150f2ddb37dde,1.0
TRAAAED128E0783FAB,It's About Time,Jamie Cullum,https://p.scdn.co/mp3-preview/9bb7b58f848a742b...,00ZCW9lUa2bqwpF6rhtaMI,"jazz, piano, chill",Jazz,2004,247000,0.565,0.507,...,1,0.0541,0.702,0.000066,0.129,0.454,152.462,4,e49ac0612b9444abf3d513e54b1cd77f6fe5ae4b,1.0


In [7]:
#music_listening_info.describe()

In [8]:
# Data Preprocessing (deleting 'spotify_preview_url', 'year', 'duration_ms', 'mode', 'time_signature' and keeping only unique track_id)
columns_to_remove = ['spotify_preview_url', 'year', 'duration_ms', 'mode', 'time_signature']
music_listening_info.drop(columns=columns_to_remove, inplace=True)

In [9]:
music_listening_info = music_listening_info[~music_listening_info.index.duplicated(keep='first')]

# Filling NaN values with 0
music_listening_info.fillna(0, inplace=True)

music_listening_info.head(5)

<ipython-input-9-f8a42ce1ef81>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  music_listening_info.fillna(0, inplace=True)


,name,artist,spotify_id,tags,genre,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,user_id,playcount
track_id,,,,,,,,,,,,,,,,,
TRAAADT12903CCC339,Andalucia,Pink Martini,16uYYyMYpiJsUC9JzZX4Zk,"jazz, french, lounge",0,0.714,0.521,2,-9.828,0.0363,0.7270,0.825000,0.074,0.683,92.530,0,0.0
TRAAAED128E0783FAB,It's About Time,Jamie Cullum,00ZCW9lUa2bqwpF6rhtaMI,"jazz, piano, chill",Jazz,0.565,0.507,9,-8.339,0.0541,0.7020,0.000066,0.129,0.454,152.462,cf7bd4b5b398b3e150cf262d79147312a69b96ac,9.0
TRAAAHD128F42635A5,I'll Be Waiting,Adele,3QEYmDt1FwOkFBFD74cEzZ,"pop, female_vocalists, singer_songwriter, brit...",0,0.548,0.843,2,-2.674,0.0374,0.0467,0.000000,0.131,0.754,160.009,33632868427bdc6134a88c9d88eefac3831e6f5f,1.0
TRAAAQO12903CD8E1C,Take Time,The Books,46lwuNGtO8mKkcFwhzGib1,"electronic, indie, experimental",0,0.720,0.643,10,-7.805,0.1000,0.5800,0.072800,0.345,0.504,90.037,f62fa69799107799d9e717eef8bf3542a4c8dcb4,1.0
TRAAAZF12903CCCF6B,Break My Stride,Matthew Wilder,0euwOdZrYl2lqo6tJl1zL5,"pop, 80s, new_wave",0,0.908,0.695,2,-7.680,0.0448,0.1180,0.000000,0.126,0.871,109.664,0,0.0


In [10]:
# Output the columns of the DataFrame
print(music_listening_info.columns)

Index(['name', 'artist', 'spotify_id', 'tags', 'genre', 'danceability',
       'energy', 'key', 'loudness', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'user_id',
       'playcount'],
      dtype='object')


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor

def train_lgbm_model(X, y):
    """
    Trains a LightGBM Regressor model on the given features (X) and target variable (y).

    Parameters:
    - X: DataFrame, features
    - y: Series, target variable

    Returns:
    - trained LGBMRegressor model
    """
    # Scaling numeric features using MinMaxScaler
    scaler = MinMaxScaler()
    X_numeric_scaled = scaler.fit_transform(X.select_dtypes(include=[np.number]))

    # Combining scaled numeric features with non-numeric features
    X_scaled = pd.concat([pd.DataFrame(X_numeric_scaled, columns=X.select_dtypes(include=[np.number]).columns, index=X.index),
                         X.select_dtypes(exclude=[np.number])], axis=1)

    # Convert categorical columns to category type
    categorical_columns = ['spotify_id', 'user_id']
    X_scaled[categorical_columns] = X_scaled[categorical_columns].astype('category')

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Initializing the LightGBM Regressor
    lgbm_model = LGBMRegressor()

    # Training the model
    lgbm_model.fit(X_train, y_train)

    return lgbm_model, scaler

def recommend_songs_for_user(target_user_id, music_listening_info, lgbm_model, scaler):
    """
    Recommends songs for a given user based on their listening history using a trained LightGBM model.

    Parameters:
    - target_user_id: str, the user for whom songs are recommended
    - music_listening_info: DataFrame, the entire music listening history dataset
    - lgbm_model: trained LightGBM Regressor model
    - scaler: MinMaxScaler object used for scaling features

    Returns:
    - DataFrame, recommended songs with additional information
    """
    # Select songs with playcount >= 5 for the given user_id
    target_songs = music_listening_info[(music_listening_info['user_id'] == target_user_id) & (music_listening_info['playcount'] >= 5)]

    # Extract unique artists from the selected songs
    unique_artists = target_songs['artist'].unique()

    # Recommend other songs by the same artists
    recommended_songs = music_listening_info[music_listening_info['artist'].isin(unique_artists) & (music_listening_info['user_id'] != target_user_id)]

    # Extract features and target variable for LightGBM
    X_recommended = recommended_songs.drop(['playcount', 'name', 'artist', 'tags', 'genre'], axis=1)
    y_recommended = recommended_songs['playcount']

    # Extracting numeric features
    numeric_features = X_recommended.select_dtypes(include=[np.number])

    # Scaling numeric features using MinMaxScaler
    X_numeric_scaled = scaler.transform(numeric_features)

    # Convert categorical columns to category type for recommended songs
    X_recommended_scaled = pd.concat([pd.DataFrame(X_numeric_scaled, columns=numeric_features.columns, index=X_recommended.index),
                                      X_recommended.select_dtypes(exclude=[np.number])], axis=1)
    categorical_columns = ['spotify_id', 'user_id']
    X_recommended_scaled[categorical_columns] = X_recommended_scaled[categorical_columns].astype('category')

    # Making predictions using the trained LightGBM model
    y_recommended_pred = lgbm_model.predict(X_recommended_scaled)

    # Adding predicted playcount as a new column to the recommended songs DataFrame
    recommended_songs['predicted_playcount'] = y_recommended_pred

    # Adding target_user_id to the recommended songs DataFrame
    recommended_songs['target_user_id'] = target_user_id

    # Sorting recommended songs by predicted playcount in descending order
    sorted_recommended_songs = recommended_songs.sort_values(by='predicted_playcount', ascending=False)

    return sorted_recommended_songs[['name', 'artist', 'tags', 'genre', 'predicted_playcount', 'target_user_id']].head(10)


X = music_listening_info.drop(['playcount', 'name', 'artist', 'tags', 'genre'], axis=1)
y = music_listening_info['playcount']


trained_model, trained_scaler = train_lgbm_model(X, y)


target_user_id = 'cf7bd4b5b398b3e150cf262d79147312a69b96ac'
recommended_songs = recommend_songs_for_user(target_user_id, music_listening_info, trained_model, trained_scaler)

print(recommended_songs)


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001408 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4152
[LightGBM] [Info] Number of data points in the train set: 40546, number of used features: 12
[LightGBM] [Info] Start training from score 1.462068
                                             name        artist  \
track_id                                                          
TRWVCVL128E07999EE                        My Yard  Jamie Cullum   
TRRITEJ128F4266ABF        It Ain't Necessarily So  Jamie Cullum   
TRRGXNI128F426938A               Well You Needn't  Jamie Cullum   
TRCHXPX12903CBFA2B          

<ipython-input-13-1dc1df571681>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_songs['predicted_playcount'] = y_recommended_pred
<ipython-input-13-1dc1df571681>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_songs['target_user_id'] = target_user_id
